In [1]:
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package
import torch.nn.functional as F
import numpy
import matplotlib.pyplot as plt

device = torch.device("cuda")

In [2]:
features = 16
# define a simple linear VAE #until now normal VAE without Beta
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
 
        # encoder 84*84 = 7’056
        self.enc0 = nn.Linear(in_features=84*84, out_features=1024)
        self.enc1 = nn.Linear(in_features=1024, out_features=512)
        self.enc2 = nn.Linear(in_features=512, out_features=features*2)
 
        # decoder 
        self.dec0 = nn.Linear(in_features=features, out_features=512)
        self.dec1 = nn.Linear(in_features=512, out_features=1024)
        self.dec2 = nn.Linear(in_features=1024, out_features=84*84)

    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    
    def encode(self, x):
        x = F.relu(self.enc0(x))
        x = F.relu(self.enc1(x))
        x = self.enc2(x)
        return x
    
    def decode(self, x):
        x = F.relu(self.dec0(x))
        x = F.relu(self.dec1(x))
        x = torch.sigmoid(self.dec2(x))
        return x

In [3]:
train_data = numpy.load('train_data100kFEB23.npy')
val_data = numpy.load('val_data20kFEB23.npy')

In [4]:
print(train_data.shape)

(101848, 84, 84)


In [5]:
BVAE = LinearVAE()
BVAE.load_state_dict(torch.load('C:/Users/erics/Documents/Programme/Bachelorarbeit/models/BTCVAE_Pong/linearB500_TC0_Lat16lr0.0001VAEMAR23'))
BVAE.to(device)

LinearVAE(
  (enc0): Linear(in_features=7056, out_features=1024, bias=True)
  (enc1): Linear(in_features=1024, out_features=512, bias=True)
  (enc2): Linear(in_features=512, out_features=32, bias=True)
  (dec0): Linear(in_features=16, out_features=512, bias=True)
  (dec1): Linear(in_features=512, out_features=1024, bias=True)
  (dec2): Linear(in_features=1024, out_features=7056, bias=True)
)

In [6]:
randomInput = torch.rand(256, 32, device=device) #32 = 2* features
print(randomInput)


tensor([[0.6069, 0.5203, 0.9740,  ..., 0.4359, 0.4476, 0.5721],
        [0.2260, 0.2239, 0.1097,  ..., 0.6532, 0.5242, 0.2523],
        [0.7325, 0.6924, 0.4129,  ..., 0.3571, 0.2408, 0.6141],
        ...,
        [0.6659, 0.1913, 0.8642,  ..., 0.9827, 0.8966, 0.6453],
        [0.0033, 0.4597, 0.3822,  ..., 0.8363, 0.6561, 0.5659],
        [0.4606, 0.1436, 0.3239,  ..., 0.9180, 0.7845, 0.5667]],
       device='cuda:0')


In [7]:
identMatrix = torch.eye(32, device = device)
print(identMatrix)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')


In [8]:
def reparameterize(mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample

In [9]:
x = identMatrix.view(-1, 2, 16) # features = 64
mu = x[:, 0, :] # the first feature values as mean
log_var = x[:, 1, :] # the other feature values as variance

Input = reparameterize(mu, log_var)
print(Input)

tensor([[ 0.7097,  0.2597, -0.1552, -0.8025,  2.0147,  1.2720, -0.3399,  0.4496,
          1.3002,  0.0120, -0.5073, -0.2735, -0.8766,  0.7312, -0.0264, -0.5131],
        [ 1.5290,  1.1887, -1.7397,  0.3381,  1.8004,  0.1829,  0.0732, -0.6500,
          1.5002, -0.2415,  0.5754, -0.6359, -0.6251,  0.2739,  1.2305,  0.8772],
        [-1.1384,  0.2913,  0.4959,  1.8156, -0.1077, -0.3647,  0.8191, -0.6895,
          0.9104,  0.6833, -0.1286,  1.6859,  0.0527,  0.9260,  0.3765,  0.1374],
        [ 0.8439,  0.6631,  2.0116,  2.6906,  0.1207,  0.2878, -1.1127,  0.0043,
         -0.5038,  0.1056, -1.8679,  0.2122,  0.7487,  0.7318, -0.3111,  1.8641],
        [ 0.1656,  0.1271, -0.3235, -0.3089,  1.8042,  0.2791, -1.1342, -0.6572,
          1.5341,  0.9469, -0.0656,  0.7499,  0.1277, -1.4564,  0.7767,  1.0568],
        [-0.5459,  0.6802,  0.7546,  1.4509,  0.3284,  0.2784, -0.4114,  0.7129,
         -0.4399, -1.3519, -0.3700, -0.4945, -0.8664,  0.8036,  1.0755, -2.1706],
        [ 1.3387, -0.4

In [10]:
recon = BVAE.decode(Input).view(-1, 84, 84)
print(recon)

tensor([[[2.5213e-06, 6.4490e-07, 1.4559e-06,  ..., 2.9511e-07,
          2.0446e-07, 7.0824e-07],
         [2.9553e-06, 1.1472e-04, 1.5889e-04,  ..., 1.1378e-04,
          2.5384e-05, 6.4773e-06],
         [4.1338e-06, 2.3661e-04, 5.1941e-04,  ..., 9.2303e-06,
          7.1965e-06, 5.0738e-07],
         ...,
         [2.3326e-06, 1.8308e-04, 1.3488e-03,  ..., 2.6168e-04,
          6.2827e-07, 1.7431e-07],
         [5.2572e-06, 4.8508e-07, 3.8024e-07,  ..., 1.7105e-06,
          8.1924e-08, 3.8548e-07],
         [4.7070e-07, 3.8107e-07, 9.9084e-08,  ..., 3.3862e-07,
          2.3618e-07, 5.2048e-06]],

        [[2.0803e-07, 5.0433e-07, 3.3684e-08,  ..., 1.1890e-07,
          2.2402e-08, 5.5955e-08],
         [5.1976e-07, 1.2857e-04, 3.3907e-06,  ..., 3.1249e-05,
          3.9247e-06, 1.7120e-06],
         [3.3938e-07, 2.2934e-04, 2.4306e-04,  ..., 1.2291e-05,
          2.0901e-07, 2.6521e-08],
         ...,
         [3.9539e-07, 4.1957e-05, 4.6219e-04,  ..., 1.4112e-04,
          2.199

In [11]:
recon = recon.detach().cpu().numpy()

In [12]:
print(recon.shape)

(32, 84, 84)


In [13]:
axes=[]
fig = plt.figure(figsize=(200,200))

for a in range(recon.shape[0]):
    axes.append( fig.add_subplot(4, 8, a+1) )
    axes[-1].set_title(str(a+1))  
    plt.imshow(recon[a])
    

plt.show()